In [1]:
import radtran
import numpy as np
incAngle=53.0 # incidence angle in degrees
 # cos of incidence angle
nlyr=80 # number of layers; cannot exceed 80; more layers require changing MAXLYR parameter in src/radtran_tau_dble.f

btemp=300.0 # skin temperature
lyrtemp=300.0*np.ones(nlyr+1) # layer temperature
lyrhgt=np.arange(nlyr+1)*.25 # layer height in km
kext=0.01*np.ones(nlyr) # extinction coefficient in km^-1
salb=0.0*np.ones(nlyr) # single scattering albedo
asym=0.0*np.ones(nlyr) # asymmetry parameter
fisot=2.7 # background radiation in K
emis=0.9 # surface emissivity at the incidence angle
ebar=0.9 # average surface emissivity 
ilambert=0 
umu=np.cos(incAngle/180*np.pi)# whether surface is lambertian surface
prnt=0 # whether to print out the results # 
tb=radtran.radtran(umu,nlyr,btemp,lyrtemp,lyrhgt,kext,salb,asym,fisot,emis,ebar,ilambert)
print(tb)


284.70587158203125


In [2]:
import lkTables as lkT
lkTables=lkT.scattTables()
print(dir(lkTables))
# GMI frequency channels
frequencies=[10.65,18.7,23.8,36.5,89.0,166.0,183.3]
#(lkTables.kextR.shape)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'asymBB', 'asymG', 'asymR', 'asymS', 'attKaBB', 'attKaG', 'attKaR', 'attKaS', 'attKuBB', 'attKuG', 'attKuR', 'attKuS', 'dmBB', 'dmg', 'dmr', 'dms', 'fh', 'fhBB', 'fhGMI', 'graupRate', 'gwc', 'kextBB', 'kextG', 'kextR', 'kextS', 'precRateBB', 'rainRate', 'rwc', 'salbBB', 'salbG', 'salbR', 'salbS', 'snowRate', 'swc', 'zKaBB', 'zKaG', 'zKaR', 'zKaS', 'zKuBB', 'zKuG', 'zKuR', 'zKuS']


There are multiple types of hydrometeors and multiple physical and electromagnetic scattering in the tables. The hydrometeor types are: rain, snow, melting snow, and graupel.  The information is stored in multiple arrays per species. For example:
#### Rain 
Properties are stored in 
* lkTables.dmR - mass mean diameter (mm), 1D
* lkTables.rainRate - rain rate (mm/h), 1D
* lkTables.rwc  - rain water content (g/m^3), 1D
* lkTables.kextR - rain extinction coefficient (km^(-1)), 2D, the second dimension is associated with one of the GMI frequencies
* lkTables.salbR - rain scattering albedo, i.e. scattering over extinction (dimensionless)
* lkTables.asymR - rain asymmetry factor (or g) the first coefficient in the Legendre expansion of the scattering function
There are multiple ways of extracting information from these tables, i.e. by reflectivity (zKuR), dm, rate or water content, by only reflectivity does not require search. 

For a quick search you can use the bisection method implemented in bisectm.py.

In [5]:
from bisectm import bisectm
# Here are some examples of how to use the bisectm function to find the electromagnetic scattering properties of rain particles

# find the entry associated with 0.1 g/m^3 in the rwc table
ind=bisectm(lkTables.rwc.data,0.1)
# you can interpolate among the entries to get exactly the same value, but the errors are negligible if you use the nearest entry
rwc_rain=lkTables.rwc.data[ind]
dm_rain=lkTables.dmr.data[ind]
rainRate=lkTables.rainRate.data[ind]

kext_rain=lkTables.kextR[ind] # extinction coefficient (7 values associated the 7 GMI frequencies)
asym_rain=lkTables.asymR[ind]
salb_rain=lkTables.salbR[ind]

print("Rain water content: ",rwc_rain," g/m^3")
print("Drop mean diameter: ",dm_rain," mm")
print("Rain rate: ",rainRate," mm/h")   
print("Extinction coefficient: ",kext_rain," km^-1")
print("Asymmetry parameter: ",asym_rain)
print("Single scattering albedo: ",salb_rain)

Rain water content:  0.100362696  g/m^3
Drop mean diameter:  1.0055434  mm
Rain rate:  1.397494  mm/h
Extinction coefficient:  [0.0042065  0.01580977 0.02741494 0.07357213 0.35188225 0.49624076
 0.50615835]  km^-1
Asymmetry parameter:  [0.03746443 0.02172826 0.01163911 0.01826801 0.19542596 0.43504894
 0.4675637 ]
Single scattering albedo:  [0.02266642 0.06159126 0.09729434 0.20200863 0.40239608 0.46004292
 0.4655948 ]


In [6]:
import netCDF4 as nc
with nc.Dataset('profiles137/saf137.nc') as f:
    qv=f['hum'][:]
    temp=f['temp'][:]
    dz=f['dz'][:]
    p=f['pap'][:]
    rain=f['rain'][:]*3600 # convert from kg/m^2/s to mm/h
    snow=f['snow'][:]*3600 # convert from kg/m^2/s to mm/h
rho=p/(287.05*temp)

In [19]:
frequencies=[10.65,18.7,23.8,36.5,89.0,166.0,183.3]
ifreq,iprof=0,10 # choose a frequency and a profile number
freq=frequencies[ifreq]
nlyr=80
hL=[0]
kext1D=np.zeros(nlyr)
asym1D=np.zeros(nlyr)
salb1D=np.zeros(nlyr)
btemp=temp[iprof,-1]
lyrtemp=[temp[iprof,-1]]
for k in range(nlyr):
    nk=-1-k
    ireturn=0
    #print(nk)
    absair,abswv = radtran.gasabsr98(freq,temp[iprof,nk],qv[iprof,nk]*rho[iprof,nk],p[iprof,nk],ireturn)
    hL.append(hL[-1]+dz[iprof,nk])
    kext_rain=0
    asym_rain=0
    salb_rain=0
    lyrtemp.append(0.5*(temp[iprof,nk]+temp[iprof,nk-1]))
    if rain[iprof,nk]>0.01:
        ind=bisectm(lkTables.rwc.data,rain[iprof,nk])
        rwc_rain=lkTables.rwc.data[ind]
        dm_rain=lkTables.dmr.data[ind]
        rainRate=lkTables.rainRate.data[ind]
        kext_rain=lkTables.kextR[ind][ifreq]
        asym_rain=lkTables.asymR[ind][ifreq]
        salb_rain=lkTables.salbR[ind][ifreq]
        absair+=rwc_rain*kext_rain
        abswv+=rwc_rain*kext_rain
    kext1D[k]=absair+abswv+kext_rain
    asym1D[k]=asym_rain*kext_rain/(kext1D[k])
    salb1D[k]=salb_rain
lyrtemp=np.array(lyrtemp)
lyrhgt=np.array(hL)/1e3 # convert to km

fisot=2.7 # background radiation in K
emis=0.9 # surface emissivity at the incidence angle
ebar=0.9 # average surface emissivity 
npol=0 # integer defining the polarization 0=H, 1=V
ws=10 # wind speed in m/s
emis,ebar=radtran.emit( freq, npol, btemp, ws, umu) # surface emissivity and average surface emissivity over oceans
ilambert=0 
umu=np.cos(incAngle/180*np.pi)# whether surface is lambertian surface
prnt=0 # whether to print out the results # 
tb=radtran.radtran(umu,nlyr,btemp,lyrtemp,lyrhgt,kext1D,salb1D,asym1D,fisot,emis,ebar,ilambert)
print(tb)

#print(hL)

270.6405029296875
